In [1]:
import json

# 파일 이름
input_filename = 'keras-team_keras_issues_4875_class.json'
output_filename = 'sorted_keras-team_keras_issues_4875_class.json'

# JSON 파일 읽기
with open(input_filename, 'r', encoding='utf-8') as file:
    data = json.load(file)

# issueObjectAnchor 기준으로 정렬하는 함수
def sort_by_issue_object_anchor(data):
    return sorted(data, key=lambda x: int(x["issueObjectAnchor"].replace("IssueSPTOText", "")))

# 정렬 수행
sorted_data = sort_by_issue_object_anchor(data)

# 정렬된 데이터를 새로운 파일에 저장
with open(output_filename, 'w', encoding='utf-8') as file:
    json.dump(sorted_data, file, ensure_ascii=False, indent=4)

print(f"정렬된 데이터가 '{output_filename}'에 저장되었습니다.")

정렬된 데이터가 'sorted_keras-team_keras_issues_4875_class.json'에 저장되었습니다.


In [2]:
# 파일 이름
issues_filename = 'sorted_keras-team_keras_issues_4875_class.json'
actions_filename = 'keras-team_keras_issues_4875_Start_End.json'
output_filename = 'merged_issues.json'

# JSON 파일 읽기
with open(issues_filename, 'r', encoding='utf-8') as file:
    issues = json.load(file)

with open(actions_filename, 'r', encoding='utf-8') as file:
    actions = json.load(file)

# 합칠 데이터 생성
merged_data = []

# 각 이슈에 대해 합치기
for issue in issues:
    issue_index = int(issue["issueObjectAnchor"].replace("IssueSPTOText", ""))
    
    # actions 리스트에서 start와 end 범위에 맞는 액션 찾기
    for action in actions:
        if action["start"] <= issue_index < action["end"]:
            merged_issue = {**issue, **action}  # issue와 action을 병합
            merged_data.append(merged_issue)
            break

# 결과를 새로운 파일에 저장
with open(output_filename, 'w', encoding='utf-8') as file:
    json.dump(merged_data, file, ensure_ascii=False, indent=4)

print(f"합쳐진 데이터가 '{output_filename}'에 저장되었습니다.")

합쳐진 데이터가 'merged_issues.json'에 저장되었습니다.


In [3]:
import pandas as pd

df = pd.read_json('merged_issues.json')

df

,issueObjectAnchor,textType,ActiononIssue,BugReproduction,ContributionandCommitment,ExpectedBehaviour,InvestigationandExploration,Motivation,ObservedBugBehaviour,PotentialNewIssuesandRequests,SocialConversation,SolutionDiscussion,SolutionUsage,TaskProgress,Workarounds,end,start,timelineCount
0,IssueSPTOText0,{'result': 'Solution Usage'},0,1,0,0,0,1,2,0,2,5,1,0,0,12,0,0
1,IssueSPTOText1,{'result': 'Motivation'},0,1,0,0,0,1,2,0,2,5,1,0,0,12,0,0
2,IssueSPTOText2,{'result': 'Observed Bug Behaviour'},0,1,0,0,0,1,2,0,2,5,1,0,0,12,0,0
3,IssueSPTOText3,{'result': 'Observed Bug Behaviour'},0,1,0,0,0,1,2,0,2,5,1,0,0,12,0,0
4,IssueSPTOText4,{'result': 'Social Conversation'},0,1,0,0,0,1,2,0,2,5,1,0,0,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,IssueSPTOText492,{'result': 'Bug Reproduction'},1,2,0,0,1,0,1,0,4,0,2,0,1,493,481,109
493,IssueSPTOText493,{'result': 'Solution Discussion'},0,1,0,0,0,0,0,0,0,1,0,0,0,495,493,110
494,IssueSPTOText494,{'result': 'Social Conversation'},0,1,0,0,0,0,0,0,0,1,0,0,0,495,493,110
495,IssueSPTOText495,{'result': 'Bug Reproduction'},0,1,0,0,0,0,0,0,0,1,0,0,0,497,495,111
